# récrire l'historique

In [1]:
# ce sera toujours notre façon de commencer
[ -f scripts/helpers.sh ] && source scripts/helpers.sh

TOP=/Users/tparment/git/flotpython-gittutorial/notebooks



## `repo-rebase`

In [2]:
cd $TOP

# pour pouvoir recommencer le scénario depuis le début
# je nettoie complètement ce qu'on a pu faire précédemment
if [ -d repo-rebase ]; then
    echo "on repart d'un directory vide"
    rm -rf repo-rebase
fi

# on le crée
mkdir repo-rebase

# on va dedans
cd repo-rebase

on repart d'un directory vide



## rappel : un commit est immutable

en informatique, on fait la distinction entre  
objets *mutables* et *immutables*

* un terme savant pour désigner les objets  
  qui peuvent changer

* un ficher par exemple est un objet mutable

* un commit est par contre **immutable**
* dit autrement, une fois qu'il est créé  
  on ne peut plus le modifier  

* mais on peut en créer un autre 

## attention aux commits poussés

avant d'aller plus loin, souvenez-vous de cette règle d'or

> ne **jamais** récrire un commit déjà publié/poussé

si quelqu'un a déjà tiré ce commit, et vous le remplacez  
par un autre, au deuxième `pull` votre collègue  
va déclencher un désordre monstrueux

exercice : voyez-vous pourquoi ?

## un repo

In [3]:
$SCRIPTS/do rebase-init
$SCRIPTS/do rebase-master-branch
$SCRIPTS/do rebase-devel-branch

Initialized empty Git repository in /Users/tparment/git/flotpython-gittutorial/notebooks/repo-rebase/.git/
[master (root-commit) 5ba161e] A
 1 file changed, 20 insertions(+)
 create mode 100644 FILE
[master 06de616] B
 1 file changed, 1 insertion(+), 1 deletion(-)
[master 4738b38] OOPS
 1 file changed, 1 insertion(+), 1 deletion(-)
Switched to a new branch 'devel'
[devel ca46d2e] D
 1 file changed, 1 insertion(+), 1 deletion(-)
[devel e7b4a3e] E
 1 file changed, 1 insertion(+), 1 deletion(-)
Switched to branch 'master'



In [4]:
# rappel:
# git l = git log --oneline --graph
git l --all

* e7b4a3e (devel) E
* ca46d2e D
| * 4738b38 (HEAD -> master) OOPS
| * 06de616 B
|/  
* 5ba161e A



## modifier le dernier commit

le cas le plus simple de récriture de l'histoire :  
vous voulez modifier le commit que vous venez de faire  
à cause d'une faute d'orthographe dans le message


```
git commit --amend
```

si vous avez ajouté des changements dans l'index entretemps,  
ils feront partie du nouveau commit 

In [5]:
git l

* 4738b38 (HEAD -> master) OOPS
* 06de616 B
* 5ba161e A



In [6]:
# récrire le dernier commit
git commit --amend --message C

[master a984a1d] C
 Date: Sat Aug 3 15:22:39 2019 +0200
 1 file changed, 1 insertion(+), 1 deletion(-)



In [7]:
git l

* a984a1d (HEAD -> master) C
* 06de616 B
* 5ba161e A



In [8]:
# le commit 'OOPS' est 
# toujours là quelque part
# mais on ne le parcourt pas
git l --all

* e7b4a3e (devel) E
* ca46d2e D
| * a984a1d (HEAD -> master) C
| * 06de616 B
|/  
* 5ba161e A



## reconstruire avec `rebase` 

un outil permettant de rejouer une suite de changements  



![](media/merge-vs-rebase.png)

`rebase` *vs* `merge` :

les deux résultats `F` et `E'`  
ont un contenu identique  
mais la topologie est  
évidemment très différente

In [9]:
# juste pour garder une référence
git branch old-devel devel

In [10]:
git l --all

* e7b4a3e (old-devel, devel) E
* ca46d2e D
| * a984a1d (HEAD -> master) C
| * 06de616 B
|/  
* 5ba161e A



In [11]:
# ça se lit comme ceci
# reconstruire la branche devel 
# au dessus de la branche master
git rebase master devel

First, rewinding head to replay your work on top of it...
Applying: D
Using index info to reconstruct a base tree...
M	FILE
Falling back to patching base and 3-way merge...
Auto-merging FILE
Applying: E



In [12]:
git l --all

* a81b4e9 (HEAD -> devel) E
* 914f564 D
* a984a1d (master) C
* 06de616 B
| * e7b4a3e (old-devel) E
| * ca46d2e D
|/  
* 5ba161e A



## F == E'

pour s'assurer qu'on a bien le même contenu qu'avec un merge

In [13]:
git checkout -b merging master
git merge old-devel --message F

Switched to a new branch 'merging'
Auto-merging FILE
Merge made by the 'recursive' strategy.
 FILE | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)



In [14]:
git l --all

*   1bf75ba (HEAD -> merging) F
|\  
| * e7b4a3e (old-devel) E
| * ca46d2e D
| | * a81b4e9 (devel) E
| | * 914f564 D
| |/  
|/|   
* | a984a1d (master) C
* | 06de616 B
|/  
* 5ba161e A



In [15]:
# pas de différence entre les deux contenus
git diff merging devel

## `pull --rebase`

souvenez-vous des cas où il faut jouer à `pull-push`  
i.e. les cas où deux personnes partent d'un même commit  
et où on l'un des deux ne peut pas pousser  
il doit d'abord faire `pull` (cf. *fast-forward*) 

dans ces cas-là si on préfère ne pas créer de diamant  
envisagez dans ce cas-là de faire un `pull --rebase`  
c'est souvent une option proposée dans les diverses UI

## `rebase -i`

on signale enfin (détails à creuser par vous-mêmes)  
le **mode interactif** de rebase :

* toujours sur des commits **non publiés**
* on peut récrire une suite de commits pour
* changer leur ordre
* en regrouper
* en enlever
* ...

ce qui est l'outil idéal pour produire un historique propre

## exercice - amend

1. créez un commit  
   modifiez le message avec `amend`
   
1. même scénario, mais vous créez une branche `bookmark`  
   juste avant le `amend` pour vérifier  
   que le premier commit est toujours présent dans le repo
   
1. créez un commit  
   ajoutez des changements dans l'index  
   faites un `amend` et constatez que l'indez est vide
   
1. utilisez `amend` pour modifier l'auteur du commit  
   voyez la doc de `commit` pour cela

## exercice - rebase 

1. créer un repo avec 5 commits init, bugfix1, feature1, bugfix2, feature2  
1. utiliser `rebase -i` pour récrire au dessus de init  
   le même contenu en seulement 2 commit feature, bugfix
1. créer une branche au niveau de init, lui appliquer le commit bugfix  
   (*hint*: voir la commande `cherry-pick`)